In [4]:
import json
import pandas as pd
import requests
import sqlite3 as db
from sqlite3 import Error

%run C:\Users\Joshua\Jupyter_Notebook_Folders\APIkeys.py

### Functions for working with SQLite

In [5]:
def create_database(db_name):
    """ 
    create an SQLite database
    """
    conn = None
    try:
        conn = db.connect(db_name)
        #print(db.version)
    except Error as e:
        print(e)
    finally:
        conn.close()
            
def create_connection(db_name):
    """ 
    create a database connection to the SQLite database
        specified by db_name
    return: Connection object or None
    """
    conn = None
    try:
        conn = db.connect(db_name)
        return conn
    except Error as e:
        print(e)

    return conn


def check_table_exists(tbl_name, db_name): 
    """
    Check if table exists
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    exists = 0
    c.execute("SELECT count(name) FROM sqlite_master WHERE type='table'")
    if c.fetchone()[0] == 0:
        print("No tables in DB.")
    else:
        try:
            c.execute("SELECT COUNT(name) FROM sqlite_master WHERE type = 'table' AND name='" + seriesID+"';")
        except Error as e:
            print(e)
        if c.fetchone()[0] == 1:
            exists = 1
            
        else:
            print("Table does not exist")        
    conn.close()
    
    return exists 
    
    
def create_table(input_df, tbl_name, db_name):
    """
    create table
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    if not check_table_exists(tbl_name ,db_name):
        try: 
            input_df.to_sql(tbl_name, conn)
            print("Created ", tbl_name, " table.")
        except ValueError as e:
            print(e)
            print("Table was not added to the DB")       
    conn.close()
    
    
def check_duplicate_data(df, tbl_name, db_name):
    """
    check if the data to be entered already exists in the table
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    df_not_dup_data = pd.DataFrame(columns = list(df.columns))
    df_dup_data = pd.DataFrame(columns = list(df.columns))
    df_inconsistent_data = pd.DataFrame(columns = list(df.columns))
    
    # Check if data already exists in DB
    for i, row in df.iterrows():
        c.execute("SELECT date, value FROM " + tbl_name + " WHERE date = ?;", (df.loc[i]["date"],))
        queryOut = c.fetchall()
        if len(queryOut) == 0:
            df_not_dup_data = pd.concat([df_not_dup_data, df.loc[[i]]])
        elif queryOut[0][1] == df.loc[i]["value"]:
            df_dup_data = pd.concat([df_dup_data, df.loc[[i]]])
            #print(seriesID, " data for ", df.iloc[i]["date"], " already exists with same value as API:", queryOut[0][1])
        elif queryOut[0][1] != df.loc[i]["value"]:
            df_inconsistent_data = pd.concat([df_inconsistent_data, df.loc[[i]]])
            #print(seriesID, " ", df.ioc[i]["date"], "DB DATA: ", queryOut[0][1], " DOES NOT MATCH INPUT DATA: ", )
    conn.close()
    
    df_not_dup_data = df_not_dup_data.sort_values(by=['date'])
    
    return df_not_dup_data, df_dup_data, df_inconsistent_data


def add_to_database(df_EntryData, tbl_name, db_name):  
    """
    Add dataframe to 
    database: db_name 
    table: tbl_name
    """
    conn = create_connection(db_name)    
    try:
        df_EntryData.to_sql(seriesID, con=conn, if_exists='append')
        print("Data added to DB: \n", df_EntryData)
    except ValueError as e:
        print(e)
        print("Data was not added to the DB")
    conn.close()
    
    
def get_max_index(tbl_name, db_name):
    '''
    Get the maximum index value of the specified tbl_name
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    try:
        c.execute("SELECT MAX([index]) FROM " + seriesID)
        max_index = c.fetchone()[0]
        return max_index
    except Error as e:
        print(e)
    finally:
        conn.close()
        
def get_max_date(tbl_name, db_name):
    '''
    Get the maximum date from the specified tbl_name
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    try:
        c.execute("SELECT date FROM " + seriesID + " WHERE [index] = (SELECT MAX([index]) FROM " + seriesID +")")
        max_date = c.fetchone()[0]
        return max_date
    except Error as e:
        print(e)
    finally:
        conn.close()

def table_to_df(tbl_name, db_name):
    conn = db.connect(db_name)
    try:
        df_fromDB = pd.read_sql_query("SELECT * FROM " + tbl_name + ";", conn)
    except Error as e:
        print(e)
    conn.close()
    
    return df_fromDB

### Get data from BLS

In [8]:
def get_BLS_data(seriesID, startyear, endyear):
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'  #this will not change
    headers = {'Content-type': 'application/json'}  #This will not changed !

    # For the key seriesid enter a list of series names you wish to download
    # For the key startyear enter the start year inside ""
    # For the key endyear enter the end year inside ""
    
    parameters = {
        "seriesid":[seriesID], 
        "startyear":str(startyear), 
        "endyear":str(endyear),
        "catalog":True, 
        "calculations":False, 
        "annualaverage":False,
        "aspects":False,
        "registrationkey":os.environ['BLS_API_key'] 
     }

    data = json.dumps(parameters) # Converts the Python dictionary to JSON

    p = requests.post(base_url, data=data, headers=headers)
    json_data = json.loads(p.text)
    
    message = ""
    if json_data['message']:
        message = "For series " + seriesID + ", no data for years: "
        for i in range(len(json_data['message'])):
            message += json_data['message'][i][-4:] + ", "
    
    return message, json_data 

In [50]:
# show all the rows in the dataframe only use this for testing
pd.set_option('display.max_rows', None)

### Create and Format Dataframe

In [22]:
def format_df_from_json(json_data):
    # Create dataframe from JSON
    df = pd.DataFrame(json_data['Results']['series'][0]['data'])

    # Format date column and sort by date
    df['date'] = df['year'] + df['period']
    df['date'] = df['date'].str.replace('M', '-')
    df['date'] = df['date'].astype('string')
    df = df.sort_values(by=['date'], ignore_index=True)

    # Set the index using the value from the max index in the db table
    df["index"] = range(1, len(df.index) +1 )
    df = df.set_index(df["index"])

    df = df.drop(columns=['footnotes', 'index'])
    df['value'] = df['value'].astype(float)
    df['description'] = json_data['Results']['series'][0]['catalog']['survey_name'] + ', ' + json_data['Results']['series'][0]['catalog']['series_title']
    
    return df

In [68]:
def align_df_db_indexes(df, seriesID, db_name):
    # If table exists set db_index_start to the max index from the table + 1
    db_index_start = 0
    if check_table_exists(seriesID, "MacroData.sqlite3") != 0:
        db_index_start = get_max_index(seriesID, "MacroData.sqlite3") + 1
    
    if db_index_start == 0:
        # return the df without alterations if db_index_start == 0
        return df
    else:
        df, df_dup_data, df_inconsistent_data = check_duplicate_data(df, seriesID, db_name)
        df_db = table_to_df(seriesID, db_name)
        df = pd.concat([df, df_db])
        df = df.sort_values(by=['date'], ignore_index=True)
        return df

In [39]:
seriesID = "CUUR0000SA0"
db_name = "MacroData.sqlite3"
message, json_data = get_BLS_data(seriesID, 1999, 2018)
message

''

In [69]:
df = format_df_from_json(json_data)
df

,year,period,periodName,value,date,description
index,,,,,,
1,1999,M01,January,164.300,1999-01,"CPI for All Urban Consumers (CPI-U), All items..."
2,1999,M02,February,164.500,1999-02,"CPI for All Urban Consumers (CPI-U), All items..."
3,1999,M03,March,165.000,1999-03,"CPI for All Urban Consumers (CPI-U), All items..."
4,1999,M04,April,166.200,1999-04,"CPI for All Urban Consumers (CPI-U), All items..."
5,1999,M05,May,166.200,1999-05,"CPI for All Urban Consumers (CPI-U), All items..."
6,1999,M06,June,166.200,1999-06,"CPI for All Urban Consumers (CPI-U), All items..."
7,1999,M07,July,166.700,1999-07,"CPI for All Urban Consumers (CPI-U), All items..."
8,1999,M08,August,167.100,1999-08,"CPI for All Urban Consumers (CPI-U), All items..."
9,1999,M09,September,167.900,1999-09,"CPI for All Urban Consumers (CPI-U), All items..."


In [70]:
df = align_df_db_indexes(df, seriesID, db_name)
df

,year,period,periodName,value,date,description,index
0,1913,M01,January,9.8,1913-01,"CPI for All Urban Consumers (CPI-U), All items...",960.0
1,1913,M02,February,9.8,1913-02,"CPI for All Urban Consumers (CPI-U), All items...",961.0
2,1913,M03,March,9.8,1913-03,"CPI for All Urban Consumers (CPI-U), All items...",962.0
3,1913,M04,April,9.8,1913-04,"CPI for All Urban Consumers (CPI-U), All items...",963.0
4,1913,M05,May,9.7,1913-05,"CPI for All Urban Consumers (CPI-U), All items...",964.0
5,1913,M06,June,9.8,1913-06,"CPI for All Urban Consumers (CPI-U), All items...",965.0
6,1913,M07,July,9.9,1913-07,"CPI for All Urban Consumers (CPI-U), All items...",966.0
7,1913,M08,August,9.9,1913-08,"CPI for All Urban Consumers (CPI-U), All items...",967.0
8,1913,M09,September,10.0,1913-09,"CPI for All Urban Consumers (CPI-U), All items...",968.0
9,1913,M10,October,10.0,1913-10,"CPI for All Urban Consumers (CPI-U), All items...",969.0


In [49]:
db_name = "MacroData.sqlite3"
df, df_dup_data, df_inconsistent_data = validate_df_db(df, seriesID, db_name)

if df_dup_data.empty and df_inconsistent_data.empty:
    add_to_database(df, seriesID, db_name)

# Deal with duplicate data
if df_dup_data.empty:
    del df_dup_data 
else:
    print("Duplicate Data: \n", df_dup_data)
    
# Deal with inconsistent data
if df_inconsistent_data.empty:
    del df_inconsistent_data 
else:
    print("DATA FROM DATAFRAME INCONSISTENT WITH DATA IN DATABASE: \n", df_inconsistent_data)

Data added to DB: 
 Empty DataFrame
Columns: [year, period, periodName, value, date, description]
Index: []


NameError: name 'df_dup_data' is not defined

In [47]:
df_cpi_not_seasonal = table_to_df(seriesID, 'MacroData.sqlite3')
df_cpi_not_seasonal

,index,year,period,periodName,value,date,description
0,0,1919,M01,January,16.500,1919-01,"CPI for All Urban Consumers (CPI-U), All items..."
1,1,1919,M02,February,16.200,1919-02,"CPI for All Urban Consumers (CPI-U), All items..."
2,2,1919,M03,March,16.400,1919-03,"CPI for All Urban Consumers (CPI-U), All items..."
3,3,1919,M04,April,16.700,1919-04,"CPI for All Urban Consumers (CPI-U), All items..."
4,4,1919,M05,May,16.900,1919-05,"CPI for All Urban Consumers (CPI-U), All items..."
...,...,...,...,...,...,...,...
1267,236,2018,M08,August,252.146,2018-08,"CPI for All Urban Consumers (CPI-U), All items..."
1268,237,2018,M09,September,252.439,2018-09,"CPI for All Urban Consumers (CPI-U), All items..."
1269,238,2018,M10,October,252.885,2018-10,"CPI for All Urban Consumers (CPI-U), All items..."
1270,239,2018,M11,November,252.038,2018-11,"CPI for All Urban Consumers (CPI-U), All items..."


In [ ]:
    # Set the index using the value from the max index in the db table
    df["index"] = range(db_index_start, db_index_start + len(df.index))
    df = df.set_index(df["index"])
